# TODO:

*   Pass tokenizer as function parameter (instead of calling global)
*   



# Setup

In [1]:
!pip install -q tf-models-official==2.3.0
!pip install transformers

In [2]:
import os
import json, pickle

import re

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from sklearn.model_selection import train_test_split

import transformers
from transformers import BertTokenizer, TFBertModel
transformers.logging.set_verbosity_error() # suppress tokenizer sentences' length warnings so to not disturbate rou beautiful loading bar oh my gosh

from keras import Model
from keras.utils import plot_model
from keras.layers import Input, Dense, Flatten

from google.colab import drive, files

In [3]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Settings


In [4]:
### PARAMETERS ###
RANDOM_STATE = 42

# Dataset creation
SEQUENCE_LIMIT = 512
STRIDE = 256

TRAIN_LIMIT = 1000  # set None for the full size
VAL_LIMIT = 500    # set None for the full size
TEST_LIMIT = 500   # set None for the full size

# Model Training
EPOCHS = 3
BATCH_SIZE = 32

# export settings
LOAD_FROM_DRIVE = True
SAVE_ON_DRIVE = False

# <s> Dataset </s>

In [5]:
try:
  with open('training_set.json') as f:
    json_data = json.load(f)
except:
  with open('/content/drive/My Drive/SQUAD/training_set.json') as f:
    json_data = json.load(f)

In [6]:
# Creates DataFrames with useful columns by unpacking 'paragraphs' column
def preprocess_df(df):
  temp = []
  title_dict = {}
  contexts = []

  for i, row in df.iterrows():
    for context in row['paragraphs']:
      contexts.append(context['context'])
      for qa in context['qas']:
        question_id = qa['id']
        question = qa['question']
        for answer in qa['answers']:
          answer_text = answer['text']
          answer_start = answer['answer_start']
          answer_end = answer_start+len(answer_text)
          temp.append([question_id, question, answer_text, answer_start, answer_end, i, len(contexts)-1])

  context_dict = dict(enumerate(contexts))
  df = pd.DataFrame(temp, columns=['question_id', 'question_text', 'answer_text', 'answer_start', 'answer_end', 'title_id', 'context_id'])
  
  return df, context_dict

# Read data from json
data = pd.json_normalize(json_data['data'])

# Split train and test
train_val, test = train_test_split(data, test_size=0.2, random_state=RANDOM_STATE)
train, val = train_test_split(train_val, test_size=0.25, random_state=RANDOM_STATE)

# Create DataFrames with useful columns
train_df, train_context_dict = preprocess_df(train)
val_df, val_context_dict = preprocess_df(val)
test_df, test_context_dict = preprocess_df(test)

train_df

question_id  ... context_id
0      5705e78f75f01819005e7734  ...          0
1      5705e78f75f01819005e7735  ...          0
2      5705e78f75f01819005e7736  ...          0
3      5705e78f75f01819005e7737  ...          0
4      5705e78f75f01819005e7738  ...          0
...                         ...  ...        ...
49933  57314ace05b4da19006bcfe2  ...      10869
49934  57314ace05b4da19006bcfe3  ...      10869
49935  57314ace05b4da19006bcfe4  ...      10869
49936  57314ace05b4da19006bcfe5  ...      10869
49937  57314ace05b4da19006bcfe6  ...      10869

[49938 rows x 7 columns]

In [7]:
# Inspect data samples
def print_squad_sample(train_data, context_dict, line_length=120, separator_length=150):

  sample = train_data.sample(frac=1).head(1)
  context = context_dict[sample['context_id'].item()]

  print('='*separator_length, '\nCONTEXT:\n', '='*separator_length)

  lines = [''.join(context[idx:idx+line_length]) for idx in range(0, len(context), line_length)]
  for l in lines:
      print(l)

  print('='*separator_length, '\nQUESTION:', ' '*(3*separator_length//4), 'ANSWER:')

  questions = train_data[train_data['context_id'] == sample['context_id'].item()]

  for idx, row in questions.iterrows():
    question = row.question_text
    answer = row.answer_text
    print(question, ' '*(3*separator_length//4-len(question)+9), answer)

print_squad_sample(train_df, train_context_dict)

CONTEXT:
The University of Kansas School of Architecture, Design, and Planning (SADP), with its main building being Marvin Hall, 
traces its architectural roots to the creation of the architectural engineering degree program in KU's School of Enginee
ring in 1912. The Bachelor of Architecture degree was added in 1920. In 1969, the School of Architecture and Urban Desig
n (SAUD) was formed with three programs: architecture, architectural engineering, and urban planning. In 2001 architectu
ral engineering merged with civil and environmental engineering. The design programs from the discontinued School of Fin
e Arts were merged into the school in 2009 forming the current School of Architecture, Design, and Planning.
QUESTION:                                                                                                                  ANSWER:
What KU school is abbreviated as SADP?                                                                                     The University of Kansa

# <s> Tokenizer </s>

In [8]:
# Load pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

print('[[CLS], [SEP]] = {}'.format(tokenizer.convert_tokens_to_ids(['[CLS]', '[SEP]'])))

# Split the sentence into tokens.
tokenized_text = tokenizer.tokenize(train_df['question_text'][0])

# Print out the tokens.
print('\nTokenized sentence:')
print(tokenized_text)

# Map the token strings to their vocabulary indeces.
indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)

# Display the words with their indices.
print('\n{:<12} {:>6}\n'.format('TOKENS', 'INDICES'))
for tup in zip(tokenized_text, indexed_tokens):
    print('{:<12} {:>6}'.format(tup[0], tup[1]))

[[CLS], [SEP]] = [101, 102]

Tokenized sentence:
['where', 'does', 'migration', 'mainly', 'occur', '?']

TOKENS       INDICES

where          2073
does           2515
migration      9230
mainly         3701
occur          5258
?              1029


In [9]:
# Returns start and end indices of sublist in list
def find_sublist(a, b):
  if len(a) == 0 or len(b) == 0:
    raise Exception("Lists with length 0")

  for i in range(len(a) - len(b) +1):
    if np.array_equal(a[i:i+len(b)], b):
      return i, i+len(b)-1

  return -1, -1

In [10]:
# Prints COOL progress bar (No, seriously, it's really cool!)
# REALLY DUDE, THIS STUFF IS GOLD
def print_progress(value, total, bar_width=100):
  perc = int(value/total*bar_width)
  rest = bar_width - perc
  print("\r{:>3} % [ {}{} ]".format(perc, perc*'■', rest*'─'), end="")

# ensure to preserve all sub-sequences of length limi-stride. 
def split_long_sequence(my_sequence, limit, stride):

  if len(my_sequence) <= limit: 
    return [my_sequence]
  
  rest = my_sequence
  split = []
  while len(rest) > limit:
    left_hand = rest[:limit]
    rest = rest[stride:]
    split.append(left_hand)
    
  split.append(rest)
  return split

In [11]:
def from_df_to_model_dict(df, context_dict, verbose=False):

  # initialize structures
  input_ids = []
  input_mask = []
  input_type_ids = []
  start_indexes = []
  end_indexes = []

  cut_counter = 0
  max_iter = len(df)
  new_entries = 0
  for i, row in df.iterrows():

    # print progress
    if (i+1) % (max_iter // 100) == 0 or i == 0:
      print_progress(i+1, max_iter)

    # encode question and context
    if verbose: print("\tTokenizing question and context...", end="")
    encoded_question = tokenizer.encode(row['question_text'])
    encoded_context = tokenizer.encode(context_dict[row['context_id']])[1:]

    # concatenate input data 
    if verbose: print("\tChecking sequence length...")
    if len(encoded_question + encoded_context) > SEQUENCE_LIMIT:
      # if the sequence is too long, split it in n subsequences of length <= SEQUENCE_LIMIT
      encoded_contexts = split_long_sequence(encoded_context,
                                             limit=SEQUENCE_LIMIT-len(encoded_question), 
                                             stride=STRIDE)
      cut_counter += 1
    else:
      encoded_contexts = [encoded_context]
    
    new_entries += len(encoded_contexts)-1
    # for each too long sequence, the context has been split in n parts. We need to process them separately, creating new entries for the input
    for context_piece in encoded_contexts:
      
      encoded_input = encoded_question + context_piece

      # create mask of ones
      ones_mask = tf.ones_like(encoded_input)

      # add padding and convert to tensor
      if verbose: print("\tPadding...", end="")
      encoded_input = tf.keras.preprocessing.sequence.pad_sequences([encoded_input], maxlen=512, padding='pre')
      encoded_input = tf.squeeze(tf.convert_to_tensor(encoded_input))

      # processing answer indices
      if verbose: print("\tProcessing answer...", end="")
      tokenized_answer = tokenizer.encode(row['answer_text'])[1: -1]
      start_idx, end_idx = find_sublist(encoded_input, tokenized_answer)

      start_indexes.append(start_idx)
      end_indexes.append(end_idx)


      # create input_type_ids
      if verbose: print("\tInput types creation...", end="")
      type_ids = tf.concat([tf.zeros_like(encoded_question, dtype=tf.int32), 
                            tf.ones_like(context_piece, dtype=tf.int32)], 
                          axis=-1)
      
      type_ids = tf.keras.preprocessing.sequence.pad_sequences([type_ids], maxlen=512, padding='pre')
      type_ids = tf.squeeze(tf.convert_to_tensor(type_ids))


      # create mask of zeros
      if verbose: print("\tMask creation...", end="")
      zeros_mask = tf.zeros(SEQUENCE_LIMIT - len(ones_mask), dtype=tf.int32)
      mask = tf.concat([zeros_mask, ones_mask], axis=-1)

      # append elements to lists
      if verbose: print("\tAppending inputs...", end="")
      input_ids.append(encoded_input)
      input_mask.append(mask)
      input_type_ids.append(type_ids)

  # save input data as dictionary
  inputs = {
    'input_ids': tf.convert_to_tensor(input_ids),
    'attention_mask': tf.convert_to_tensor(input_mask),
    'token_type_ids': tf.convert_to_tensor(input_type_ids)
    }

  assert len(input_ids) == len(df) + new_entries, f"ATTENTION:\nci sono {len(input_ids)} input_ids, MA {len(df)} righe nel df e {new_entries} nuove entries"
  return inputs, start_indexes, end_indexes

# <s> Building Dataset </s>

In [12]:
if not LOAD_FROM_DRIVE:
  print("Building training set...")
  train_inputs, train_starts_ids, train_ends_ids = from_df_to_model_dict(train_df[:TRAIN_LIMIT], train_context_dict)

  print("\n\nBuilding validation set...")
  val_inputs, val_starts_ids, val_ends_ids = from_df_to_model_dict(val_df[:VAL_LIMIT], val_context_dict)

  print("\n\nBuilding test set...")
  test_inputs, test_starts_ids, test_ends_ids = from_df_to_model_dict(test_df[:TEST_LIMIT], test_context_dict)

# <s> Index to categorical <s>


In [13]:
if not LOAD_FROM_DRIVE: 
  # Replace -1 (answer not found) with 0
  train_starts_ids = [val if val != -1 else 0 for val in train_starts_ids]
  train_ends_ids = [val if val != -1 else 0 for val in train_ends_ids]

  val_starts_ids = [val if val != -1 else 0 for val in val_starts_ids]
  val_ends_ids = [val if val != -1 else 0 for val in val_ends_ids]

  test_starts_ids = [val if val != -1 else 0 for val in test_starts_ids]
  test_ends_ids = [val if val != -1 else 0 for val in test_ends_ids]

  # from ints to categorical
  train_starts_cat = tf.keras.utils.to_categorical(train_starts_ids, num_classes=SEQUENCE_LIMIT)
  train_ends_cat = tf.keras.utils.to_categorical(train_ends_ids, num_classes=SEQUENCE_LIMIT)

  val_starts_cat = tf.keras.utils.to_categorical(val_starts_ids, num_classes=SEQUENCE_LIMIT)
  val_ends_cat = tf.keras.utils.to_categorical(val_ends_ids, num_classes=SEQUENCE_LIMIT)

  test_starts_cat = tf.keras.utils.to_categorical(test_starts_ids, num_classes=SEQUENCE_LIMIT)
  test_ends_cat = tf.keras.utils.to_categorical(test_ends_ids, num_classes=SEQUENCE_LIMIT)

  # Inspect data
  assert len(train_starts_ids) == len(train_ends_ids) == len(train_inputs['input_ids']) == len(train_inputs['attention_mask']) == len(train_inputs['token_type_ids'])
  assert len(val_starts_ids) == len(val_ends_ids) == len(val_inputs['input_ids']) == len(val_inputs['attention_mask']) == len(val_inputs['token_type_ids'])
  assert len(test_starts_ids) == len(test_ends_ids) == len(test_inputs['input_ids']) == len(test_inputs['attention_mask']) == len(test_inputs['token_type_ids'])

  print("Shapes")
  print("\nTrain starts:\t", train_starts_cat.shape)
  print("Train ends:\t", train_ends_cat.shape)
  print("\nVal starts:\t", val_starts_cat.shape)
  print("Val ends:\t", val_ends_cat.shape)
  print("\nTest starts:\t", test_starts_cat.shape)
  print("Test ends:\t", test_ends_cat.shape)

# <s>  Save inputs dataset <s>

In [14]:
if SAVE_ON_DRIVE:

  # TRAIN
  with open("/content/drive/MyDrive/SQUAD/train_inputs.p", "wb") as f:
    pickle.dump(train_inputs, f)

  with open("/content/drive/MyDrive/SQUAD/train_starts_ids.p", "wb") as f:
    pickle.dump(train_starts_ids, f)

  with open("/content/drive/MyDrive/SQUAD/train_ends_ids.p", "wb") as f:
    pickle.dump(train_ends_ids, f)

  with open("/content/drive/MyDrive/SQUAD/train_starts_cat.p", "wb") as f:
    pickle.dump(train_starts_cat, f)

  with open("/content/drive/MyDrive/SQUAD/train_ends_cat.p", "wb") as f:
    pickle.dump(train_ends_cat, f)


  # VALIDATION
  with open("/content/drive/MyDrive/SQUAD/val_inputs.p", "wb") as f:
    pickle.dump(val_inputs, f)

  with open("/content/drive/MyDrive/SQUAD/val_starts_ids.p", "wb") as f:
    pickle.dump(val_starts_ids, f)

  with open("/content/drive/MyDrive/SQUAD/val_ends_ids.p", "wb") as f:
    pickle.dump(val_ends_ids, f)

  with open("/content/drive/MyDrive/SQUAD/val_starts_cat.p", "wb") as f:
    pickle.dump(val_starts_cat, f)

  with open("/content/drive/MyDrive/SQUAD/val_ends_cat.p", "wb") as f:
    pickle.dump(val_ends_cat, f)


  # TEST
  with open("/content/drive/MyDrive/SQUAD/test_inputs.p", "wb") as f:
    pickle.dump(test_inputs, f)

  with open("/content/drive/MyDrive/SQUAD/test_starts_ids.p", "wb") as f:
    pickle.dump(test_starts_ids, f)

  with open("/content/drive/MyDrive/SQUAD/test_ends_ids.p", "wb") as f:
    pickle.dump(test_ends_ids, f)

  with open("/content/drive/MyDrive/SQUAD/test_starts_cat.p", "wb") as f:
    pickle.dump(test_starts_cat, f)

  with open("/content/drive/MyDrive/SQUAD/test_ends_cat.p", "wb") as f:
    pickle.dump(test_ends_cat, f)

# Load train and test inputs datasets

In [15]:
if LOAD_FROM_DRIVE:
  ## TRAIN
  with open("/content/drive/MyDrive/SQUAD/train_inputs.p", "rb") as f:
    train_inputs = pickle.load(f)

  with open("/content/drive/MyDrive/SQUAD/train_starts_ids.p", "rb") as f:
    train_starts_ids = pickle.load(f)

  with open("/content/drive/MyDrive/SQUAD/train_ends_ids.p", "rb") as f:
    train_ends_ids = pickle.load(f)

  with open("/content/drive/MyDrive/SQUAD/train_starts_cat.p", "rb") as f:
    train_starts_cat = pickle.load(f)

  with open("/content/drive/MyDrive/SQUAD/train_ends_cat.p", "rb") as f:
    train_ends_cat = pickle.load(f)


  ## VALIDATION
  with open("/content/drive/MyDrive/SQUAD/val_inputs.p", "rb") as f:
    val_inputs = pickle.load(f)

  with open("/content/drive/MyDrive/SQUAD/val_starts_ids.p", "rb") as f:
    val_starts_ids = pickle.load(f)

  with open("/content/drive/MyDrive/SQUAD/val_ends_ids.p", "rb") as f:
    val_ends_ids = pickle.load(f)

  with open("/content/drive/MyDrive/SQUAD/val_starts_cat.p", "rb") as f:
    val_starts_cat = pickle.load(f)

  with open("/content/drive/MyDrive/SQUAD/val_ends_cat.p", "rb") as f:
    val_ends_cat = pickle.load(f)


  ## TEST
  with open("/content/drive/MyDrive/SQUAD/test_inputs.p", "rb") as f:
    test_inputs = pickle.load(f)

  with open("/content/drive/MyDrive/SQUAD/test_starts_ids.p", "rb") as f:
    test_starts_ids = pickle.load(f)

  with open("/content/drive/MyDrive/SQUAD/test_ends_ids.p", "rb") as f:
    test_ends_ids = pickle.load(f)

  with open("/content/drive/MyDrive/SQUAD/test_starts_cat.p", "rb") as f:
    test_starts_cat = pickle.load(f)

  with open("/content/drive/MyDrive/SQUAD/test_ends_cat.p", "rb") as f:
    test_ends_cat = pickle.load(f)

In [16]:
print("Shapes")
print("\nTrain starts:\t", train_starts_cat.shape)
print("Train ends:\t", train_ends_cat.shape)
print("\nVal starts:\t", val_starts_cat.shape)
print("Val ends:\t", val_ends_cat.shape)
print("\nTest starts:\t", test_starts_cat.shape)
print("Test ends:\t", test_ends_cat.shape)

assert train_starts_cat.shape == train_ends_cat.shape
assert val_starts_cat.shape == val_ends_cat.shape
assert test_starts_cat.shape == test_ends_cat.shape

Shapes

Train starts:	 (49996, 512)
Train ends:	 (49996, 512)

Val starts:	 (20098, 512)
Val ends:	 (20098, 512)

Test starts:	 (17630, 512)
Test ends:	 (17630, 512)


# Build model

In [17]:
from keras.layers import Reshape, GlobalAveragePooling1D

In [18]:
@tf.autograph.experimental.do_not_convert
def build_model():

  # Define Input layers
  input_ids_layer = Input(shape = (512,), dtype='int64')
  attention_mask_layer = Input(shape = (512,), dtype='int64')
  token_ids_layer = Input(shape = (512,), dtype='int64')

  # Load Bert model
  bert_model = TFBertModel.from_pretrained('bert-base-uncased')
  
  # Set bert layers to not-trainable
  for layer in bert_model.layers:
    layer.trainable = False
  
  # Initialize bert model
  bert = bert_model(input_ids = input_ids_layer,
                    attention_mask = attention_mask_layer,
                    token_type_ids = token_ids_layer,
                    return_dict = False,
                    output_attentions = False,
                    output_hidden_states = False
                    ).last_hidden_state

  # flat = Flatten()(bert)  
  
  # # Define output classifiers
  # start_classifier = Dense(units=512, activation="softmax", name="Start_classifier")(flat)
  # end_classifier = Dense(units=512, activation="softmax", name="End_classifier")(flat)

  # pooler = GlobalAveragePooling1D()(bert)
  classifier = Dense(units=1,  name="classifier")(bert)

  # Build model
  model = Model(inputs=[input_ids_layer, attention_mask_layer, token_ids_layer], 
                outputs=classifier)
  
  model.summary()
  return model

model = build_model()
plot_model(model, show_shapes=True)

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 512)]        0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 512)]        0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            [(None, 512)]        0                                            
__________________________________________________________________________________________________
tf_bert_model (TFBertModel)     TFBaseModelOutputWit 109482240   input_1[0][0]                    
                                                                 input_2[0][0]                

In [19]:
def compute_loss(labels, logits):
  
  loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()
  
  start_logits = logits[:, :, 0]
  end_logits = logits[:, :, 1]

  start_pred = np.argmax(start_logits, axis=1)
  end_pred = np.argmax(end_logits, axis=1)

  start_labels = labels[0, :, 0]
  end_labels = labels[0, :, 1]

  start_loss = loss_fn(start_labels, start_logits)
  end_loss = loss_fn(end_labels, end_logits)

  return start_loss + end_loss

In [20]:
# Define model parameters
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
# loss = compute_loss
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

# Train

In [21]:
# Fit model with train set
model_fit = model.fit(x = [train_inputs['input_ids'][:TRAIN_LIMIT],
                           train_inputs['attention_mask'][:TRAIN_LIMIT],
                           train_inputs['token_type_ids'][:TRAIN_LIMIT]],
                      # y = np.transpose(np.array(list(zip(train_starts_cat, train_ends_cat))), axes = (0, 2, 1)),
                      # y = np.dstack((train_starts_ids, train_ends_ids))[0],
                      y = np.array(train_starts_ids[:TRAIN_LIMIT], dtype=np.float32),
                       
                      batch_size = BATCH_SIZE,
                       
                      epochs = EPOCHS, 
                      
                      validation_data=([val_inputs['input_ids'][:VAL_LIMIT],
                                        val_inputs['attention_mask'][:VAL_LIMIT],
                                        val_inputs['token_type_ids'][:VAL_LIMIT]], 
                                        # np.transpose(np.array(list(zip(val_starts_cat, val_ends_cat))), axes = (0, 2, 1))
                                        # np.dstack((val_starts_ids, val_ends_ids))[0]
                                        #np.array(val_starts_ids, dtype=np.float32)
                                        np.array(val_starts_ids[:VAL_LIMIT], dtype=np.float32)
                                       )
                      )

Epoch 1/3
32/32 [==============================] - 44s 1s/step - loss: 6.0920 - accuracy: 0.0233 - val_loss: 6.0269 - val_accuracy: 0.0100
Epoch 2/3
32/32 [==============================] - 33s 1s/step - loss: 5.9958 - accuracy: 0.0164 - val_loss: 5.9645 - val_accuracy: 0.0100
Epoch 3/3
32/32 [==============================] - 33s 1s/step - loss: 5.9657 - accuracy: 0.0151 - val_loss: 5.9055 - val_accuracy: 0.0120


# Test

In [32]:
# Predict test set
pred = model.predict([test_inputs["input_ids"][:TEST_LIMIT], 
                      test_inputs["attention_mask"][:TEST_LIMIT], 
                      test_inputs["token_type_ids"][:TEST_LIMIT]])

print("Prediction shape: ", pred.shape)

Prediction shape:  (500, 512, 1)


In [33]:
# prediction reshaping
pred_starts_ids = np.argmax(pred[:, :, 0], axis=1)


In [34]:
import sklearn

assert pred_starts_ids.shape == np.array(test_starts_ids[:TEST_LIMIT]).shape, \
    f"\nPred shape:\t{pred_starts_ids.shape}\nLabels shape:\t{np.array(test_starts_ids[:TEST_LIMIT]).shape}"

print("test start accuracy:", sklearn.metrics.accuracy_score(test_starts_ids[:TEST_LIMIT], pred_starts_ids))
print("test start f1-score:", sklearn.metrics.f1_score(test_starts_ids[:TEST_LIMIT], pred_starts_ids, average="macro"))

test start accuracy: 0.018
test start f1-score: 0.009191919191919192


In [36]:
def get_text_from_token_ids(context, start_id, end_id):
  return tokenizer.decode(context[start_id:end_id])

# we are only using starts, ends are ignored
pred_ends_ids = [None for _ in pred_starts_ids]

boom_baby = 0
for i, row in test_df.iterrows():
  if i == TEST_LIMIT: break
  true_answer = row["answer_text"]
  predicted_answer = get_text_from_token_ids(test_inputs["input_ids"][i], pred_starts_ids[i], pred_ends_ids[i])

  if true_answer == predicted_answer:
    boom_baby += 1

  if i % 100 == 0:
    print("\n\nQuestion: \n", row["question_text"])
    print("Context: \n", test_context_dict[row["context_id"]])

    print(f"True answer: \n\tids {(test_starts_ids[i], test_ends_ids[i])}\t\ttext: {row['answer_text']}")
    print(f"Our answer: \n\tids {(pred_starts_ids[i], pred_ends_ids[i])}\t\ttext: {predicted_answer}")

    print("\n", "-"*50)



Question: 
 What is the Latin for Order of Preachers?
Context: 
 The Order of Preachers (Latin: Ordo Praedicatorum, hence the abbreviation OP used by members), more commonly known after the 15th century as the Dominican Order or Dominicans, is a Roman Catholic religious order founded by the Spanish priest Saint Dominic de Guzman in France and approved by Pope Honorius III (1216–27) on 22 December 1216. Membership in this "mendicant" order includes friars, nuns, active sisters, and lay or secular Dominicans (formerly known as tertiaries, though recently there has been a growing number of Associates, who are unrelated to the tertiaries) affiliated with the order.
True answer: 
	ids (387, 393)		text: Ordo Praedicatorum
Our answer: 
	ids (459, None)		text: mendicant " order includes friars, nuns, active sisters, and lay or secular dominicans ( formerly known as tertiaries, though recently there has been a growing number of associates, who are unrelated to the tertiaries ) affiliated with

In [ ]:
print(f"Total matches\t{boom_baby} / {TEST_LIMIT} \t({round(boom_baby / TEST_LIMIT, 3)*100} %)")

In [ ]:
import shutil

with open("results_2101_start_pred.txt", "w") as f:
  f.write(f"\nTEST start accuracy: {sklearn.metrics.accuracy_score(val_starts_ids[:TEST_LIMIT], pred_start_ids)}")
  f.write(f"\nTEST start f1-score: {sklearn.metrics.f1_score(val_starts_ids[:TEST_LIMIT], pred_start_ids, average='macro')}")    
  f.write(f"\nTotal matches\t{boom_baby} / {TEST_LIMIT} \t({round(boom_baby / TEST_LIMIT, 3)*100} %)")
  f.write(f"\n\n{str(pred_start_ids)}")

# copy on drive
shutil.copy("results_2101_start_pred.txt", "/content/drive/MyDrive/SQUAD/results_2101_start_pred.txt")
